# Predictive Analytics

```{note} hi
This is a note admonition
```

```{warning} hello
This is a warning admonition
```

```{tip}
This is a tip admonition
```

```{caution}
This is a caution admonition
```

```{attention}
This is an attention admonition
```

```{danger}
This is a danger admonition
```

```{error}
This is an error admonition
```

```{hint}
This is a hint admonition
```

```{important}
This is an important admonition
```

```{seealso}
This is a seealso admonition
```

:::{admonition} What you should already know In order to complete this tutorial, you should be relatively familiar with using the command line, as well as using a text editor. :::

:::{admonition} A note for Windows users :class: warning

Jupyter Book is now also tested against Windows OS 😀

However, there is a known incompatibility for notebook execution when using Python 3.8.

See :::